In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
import random

In [2]:
from iexfinance.stocks import Stock

In [3]:
stocks= pd.read_csv('sp_500_stocks.csv')
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
486,YUM
487,ZBH
488,ZBRA
489,ZION


In [4]:
from kamka import IEX_CLOUD_API_TOKEN

In [5]:
from json import JSONDecodeError

symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

try:
    data = requests.get(api_url).json()
    print(data)
except JSONDecodeError as e:
    print(f"Failed to decode JSON: {e}")



{'avgTotalVolume': 47405150, 'calculationPrice': 'close', 'change': -1.1, 'changePercent': -0.00569, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 192.42, 'iexCloseTime': 1701954918288, 'iexLastUpdated': 1701954918288, 'iexMarketPercent': 0.00011451911514897028, 'iexOpen': 192.42, 'iexOpenTime': 1701954918288, 'iexRealtimePrice': 192.42, 'iexRealtimeSize': 2, 'iexVolume': 12, 'lastTradeTime': 1701896399996, 'latestPrice': 192.32, 'latestSource': 'Close', 'latestTime': 'December 6, 2023', 'latestUpdate': 1701896400402, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2991105264640, 'oddLotDelayedPrice': None, 

In [14]:
my_cols= ['Ticker','Market Capital','Price','Number of shares to buy']

In [15]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [16]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings= []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe= pd.DataFrame(columns= my_cols)

for symbol_string in symbol_strings:
    batch_api_url= f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data= requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        price = data[symbol]['quote']['latestPrice']
        mcap = data[symbol]['quote']['marketCap']
        final_dataframe.loc[len(final_dataframe)] = [symbol, mcap, price, 'NA']
final_dataframe


,Ticker,Market Capital,Price,Number of shares to buy
0,A,37717425362,128.91,NA
1,AAL,8809726614,13.48,NA
2,AAP,3282129768,55.16,NA
3,AAPL,2991105264640,192.32,NA
4,ABBV,258439367686,146.38,NA
...,...,...,...,...
486,YUM,34985268813,124.81,NA
487,ZBH,24268929968,116.13,NA
488,ZBRA,12024824085,234.13,NA
489,ZION,5503727883,37.15,NA


In [ ]:
moneyToInvest= float(input("Enter the money you want to invest."))
moneyPerStock= moneyToInvest/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(moneyPerStock / final_dataframe['Price'][i])

final_dataframe

In [22]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [28]:
with pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter') as writer:
    
    final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Recommended Trades']
    column_settings = [{'header': column} for column in final_dataframe.columns]
    worksheet.add_table(0, 0, final_dataframe.shape[0], final_dataframe.shape[1] - 1, {'columns': column_settings})